In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
# Load
df = pd.read_csv("5G_NIDD_multiclass_clean.csv", low_memory=False)

print("Original shape:", df.shape)

# Target
y = df['Label']
X = df.drop(columns=['Label', 'Attack Type', 'Attack Tool'], errors='ignore')

# Remove obvious non-learning columns
drop_cols = [
    'SrcMac','DstMac','SrcAddr','DstAddr','StartTime','LastTime',
    'SrcOui','DstOui'
]

X = X.drop(columns=[c for c in drop_cols if c in X.columns], errors='ignore')

# Keep only numeric features
X = X.select_dtypes(include=[np.number])

print("After numeric selection:", X.shape)


Original shape: (1215890, 112)
After numeric selection: (1215890, 86)


In [ ]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)


In [ ]:
selector = SelectKBest(score_func=f_classif, k=36)
X_selected = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 1  7 13 14 19 20 21 22 23 24 25 26 27 34 35 36 37 48 49 50 51 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 77 78 79 80] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Selected Features: ['Rank', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max', 'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts', 'DstPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcWin', 'DstWin', 'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']


In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

num_classes = len(np.unique(y_encoded))
print("Classes:", num_classes)


Classes: 20


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)


In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)   # FIT ONLY TRAIN
X_test  = scaler.transform(X_test)        # TRANSFORM TEST


In [ ]:
X_train = X_train.reshape(-1, 36, 1)
X_test  = X_test.reshape(-1, 36, 1)


In [ ]:
def MobileNetV1(drop_rate=0.5, dense_units=256, width_mult=1.0):

    inp = Input(shape=(36,1))

    x = Reshape((36,1,1))(inp)

    # Initial conv
    x = Conv2D(int(32*width_mult),(3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise Block 1
    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(int(64*width_mult),(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise Block 2
    x = DepthwiseConv2D((3,3),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(int(128*width_mult),(1,1),padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Pooling
    x = GlobalAveragePooling2D()(x)

    # Classifier
    x = Dense(dense_units, activation="relu")(x)
    x = Dense(dense_units//2, activation="relu")(x)
    x = Dropout(drop_rate)(x)

    out = Dense(num_classes, activation="softmax")(x)

    model = Model(inp, out)
    return model


In [ ]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):

    def loss(y_true, y_pred):

        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=num_classes)

        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

        pt = tf.exp(-ce)
        focal = alpha * tf.pow(1 - pt, gamma) * ce

        return tf.reduce_mean(focal)

    return loss


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

class_weights = dict(zip(classes, class_weights))
print(class_weights)


{np.int64(0): np.float64(0.64811172410117), np.int64(1): np.float64(0.6491671115856914), np.int64(2): np.float64(3.325965944060726), np.int64(3): np.float64(4.20650406504065), np.int64(4): np.float64(37.819284603421465), np.int64(5): np.float64(44.74296228150874), np.int64(6): np.float64(1.3619983757596124), np.int64(7): np.float64(4.309374446216552), np.int64(8): np.float64(5.309563318777292), np.int64(9): np.float64(5.274438781043271), np.int64(10): np.float64(1.9601644365629534), np.int64(11): np.float64(4.803516049382716), np.int64(12): np.float64(5.220652640618291), np.int64(13): np.float64(5.217292426517915), np.int64(14): np.float64(1.5948189926547744), np.int64(15): np.float64(1.9197000197355436), np.int64(16): np.float64(0.12998123867505493), np.int64(17): np.float64(0.21242149215139894), np.int64(18): np.float64(6.053721682847897), np.int64(19): np.float64(5.8995147986414365)}


In [ ]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):

    model = MobileNetV1(
        drop_rate = hp.Choice("dropout",[0.3,0.5,0.6]),
        dense_units = hp.Choice("dense",[128,256,512]),
        width_mult = hp.Choice("width",[0.75,1.0,1.25])
    )

    lr = hp.Choice("lr",[1e-2,1e-3,1e-4])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=focal_loss(gamma=2, alpha=0.25),
        metrics=["accuracy"]
    )

    return model


tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=7,
    factor=3,
    directory="tuning_mobilenet",
    project_name="5g_mobilenet_only"
)

# ---- Tune on subset ----
sample_idx = np.random.choice(len(X_train), size=int(len(X_train)*0.25), replace=False)
X_tune = X_train[sample_idx]
y_tune = y_train[sample_idx]

stop_early = EarlyStopping(monitor='val_loss', patience=3)

tuner.search(
    X_tune, y_tune,
    validation_split=0.2,
    epochs=10,
    batch_size=512,
    callbacks=[stop_early],
    verbose=1
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hps.values)


Trial 10 Complete [00h 01m 00s]
val_accuracy: 0.7603009939193726

Best val_accuracy So Far: 0.8472942113876343
Total elapsed time: 00h 07m 57s
Best Hyperparameters:
{'dropout': 0.3, 'dense': 512, 'width': 1.0, 'lr': 0.01, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [ ]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=512,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(patience=8, restore_best_weights=True),
        ReduceLROnPlateau(patience=4)
    ]
)


Epoch 1/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 46s 22ms/step - accuracy: 0.6911 - loss: 0.1079 - val_accuracy: 0.7689 - val_loss: 0.0767 - learning_rate: 0.0100
Epoch 2/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8469 - loss: 0.0419 - val_accuracy: 0.8327 - val_loss: 0.0427 - learning_rate: 0.0100
Epoch 3/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8537 - loss: 0.0380 - val_accuracy: 0.8535 - val_loss: 0.0415 - learning_rate: 0.0100
Epoch 4/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8584 - loss: 0.0361 - val_accuracy: 0.8227 - val_loss: 0.0447 - learning_rate: 0.0100
Epoch 5/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8634 - loss: 0.0341 - val_accuracy: 0.8502 - val_loss: 0.0406 - learning_rate: 0.0100
Epoch 6/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8672 - loss: 0.0329 - val_accuracy: 0.8718 - val_loss: 0.0298 - learning_rate: 0.0100
Epoch 7/10
1710/1710 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accura

In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))


7600/7600 ━━━━━━━━━━━━━━━━━━━━ 30s 4ms/step
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     18761
           1       0.96      0.92      0.94     18730
           2       0.39      0.49      0.44      3656
           3       0.43      0.88      0.57      2890
           4       0.30      0.75      0.43       322
           5       0.41      0.14      0.21       271
           6       0.93      0.75      0.83      8927
           7       0.76      0.54      0.63      2822
           8       0.99      0.87      0.93      2290
           9       1.00      0.87      0.93      2305
          10       0.79      0.83      0.81      6203
          11       0.56      0.50      0.53      2531
          12       0.27      0.01      0.03      2329
          13       0.48      0.88      0.62      2331
          14       0.96      0.92      0.94      7624
          15       0.98      0.96      0.97      6334
          16       0.98      0.85    